# Bioequivalence

Bioequivalence.jl is a package for performing bioequivalence analysis part of the Pumas (PharmaceUtical Modeling And Simulation) ecosystem for developing, simulating, fitting, and analyzing pharmaceutical models.

This document provides a how-to and technical details for performing bioequivalence analysis using the software.

## How to obtain Bioequivalence.jl?

Bioequivalence.jl is covered under the [Julia Computing EULA](https://juliacomputing.com/eula) and distributed through via [JuliaPro](https://juliacomputing.com/products/juliapro.html) from Julia Computing Inc.

Using JuliaPro, one can install the Bioequivalence.jl package using the Julia package manager.

```
using Pkg;
Pkg.add("Bioequivalence")
```

## Getting Started

To load the package, use

In [ ]:
using Bioequivalence

The Bioequivalence package provides two functions: `read_be` and `generate_design`.

`read_be` is an alias for `BioequivalenceStudy` which takes a table with data from a bioequivalence study and other optional arguments and returns the output of the analysis. This function tries to be quite smart and flexible. For most cases, it should work how you want it to work, but allows for users to overwrite default behavior.

The other function provided by Bioequivalence.jl is `generate_design` which allows to generate data according to some bioequivalence study design.

### Setting up the environment

There are a few packages that can be helpful when using Bioequivalence.jl.

- CSV.jl is a package for importing text files (e.g., comma-separated values or CSV)
- DataFrames.jl is a package that provides a table representation for data
- StatsBase.jl is a package that provides a common way to query statistical models

In order to install any of these packages, one can use the package manager.

For example to install CSV.jl,

```
julia> using Pkg;
julia> Pkg.add("CSV")
```

We can load the packages through,

In [ ]:
using CSV, DataFrames, StatsBase, Bioequivalence

We always recommend using a local environment for each application which allows for reproducibility. For learning more about environments, we recommend checking out the Julia [documentation](https://julialang.github.io/Pkg.jl/v1/environments/#Creating-your-own-projects-1).

### Nonparametric

Nonparametric designs are included for analysis of endpoints such as Tmax.

We can load data from a bioequivalence study by using the `CSV.read` function and giving itt the path to a file.

In this case, we will use some example/validation datasets included with Bioequivalence.jl.

This dataset is available from Patterson and Jones (2006).

In [ ]:
PJ31 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Nonparametric/PJ2006_3_1.tsv"))
first(PJ31, 6)

We have now assigned the data to the variable `PJ31`.

The average bioequivalence analysis can be requested through `read_be`.

In [ ]:
Tmax = read_be(PJ31, :Tmax)

The first argument is the data and the second argument is which variable should be used as the endpoint.

Variable names `DataFrame`s are symbols.

When the name of the endpoint matches some form of Tmax (case insensitive) it defaults to nonparametric.

Otherwise, it will attempt a parametric design. This can be overwriten through the `nonparametric` argument.

Nonparametric analysis uses the Wilcoxon signed rank test.

In [ ]:
Tmax.model

Other endpoints can potentially be analyzed through a nonparametric method as well

In [ ]:
PJ46 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Williams/PJ2006_4_6.tsv"))
first(PJ46, 6)

In [ ]:
NP = read_be(PJ46, nonparametric = true)

### Parallel

Consider a parallel design dataset with balance between treatment groups such as Clayton and Leslie ([1981](https://doi.org/10.1177/030006058100900608)).

In [ ]:
ClaytonandLeslie1981 = CSV.read(string(dirname(pathof(Bioequivalence)),
                                       "/../data/Parallel/FSL2015_1.tsv"))
first(ClaytonandLeslie1981, 6)

The bioequivalence analysis can be requested through `BioequivalenceStudy` or its alias `read_be`.

In [ ]:
# notice it defaults to the AUC endpoint
Parallel = read_be(ClaytonandLeslie1981) # Same as read_be(ClaytonandLeslie1981, :AUC)

The output shows the design, statistical model, and result.

One can access the statistical model directly through

In [ ]:
Parallel.model

The results can be accessed through

In [ ]:
Parallel.result

or calling `coeftable` on the object

In [ ]:
coeftable(Parallel)

Notice that the results are validated with those reported in Fuglsang, Schütz, and Labes ([2015](https://doi.org/10.1208/s12248-014-9704-6)). The Geometric Means Ratio (GMR) has a point estimate of (48.58) and a 90% confidence interval of (26.78, 88.14). These are the values reported in the study obtained with various statistical packages with the Welch correction.

### Crossover

The most common bioequivalence design is perhaps the 2x2 crossover (RT|TR). Consider a dataset from Schütz, Labes, and Fuglsang ([2014](https://doi.org/10.1208/s12248-014-9661-0)) which has been simulated with an extreme range in raw data, outliers, and imbalance between sequences and a large number of subjects.

In [ ]:
SLF2014 = CSV.read(string(dirname(pathof(Bioequivalence)),
                          "/../data/2S2P/SLF2014_8.tsv"))
first(SLF2014, 6)

The bioequivalence analysis can be requested through `BioequivalenceStudy`

In [ ]:
# notice it defaults to the AUC endpoint
Crossover = read_be(SLF2014)

As with other designs one can access the specific elements of the models use in the analysis.

In [ ]:
loglikelihood(Crossover.model.model)

The results are obtained through,

In [ ]:
Crossover.result

Crossover 2x2 designs are validated with Schütz, Labes, and Fuglsang ([2014](https://doi.org/10.1208/s12248-014-9661-0)) and Patterson and Jones (2006). In this example, the estimate for the GMR is (93.42) with a 90% confidence interval of (86.81, 100.55).

### Balaam

The Balaam design (RR|RT|TR|TT) is explored with a dataset from Chow and Liu (2009).

In [ ]:
ChowandLiu2009 = CSV.read(string(dirname(pathof(Bioequivalence)),
                                 "/../data/Balaam/CL2009_9_2_1.tsv"))
first(ChowandLiu2009, 6)

The data has the same specification as other crossover studies.

The analysis follows similarly as well.

In [ ]:
# notice it defaults to the AUC endpoint
Balaam = read_be(ChowandLiu2009)

We can also obtain the results for the intra-subject variabilities through

In [ ]:
Balaam.model.σ

Designs with repeated measures include rescaled parameter estimates and variance analysis.

The rescaled parameter estimates are especially of interest when the reference drug is a highly variable drug (HVD) or a narrow therapeutic index (NTI) drug (Haidar et al. 2008).

The default values are:

- regulatory constant for reference-scaled estimates: 0.1
- auxiliary parameter for reference-scaled estimates: 1.11

These maybe be overwriten by passing the keyword arguments (see the docstring of `read_be` for more details).

### Dual

Consider the example 4.1 in Patterson and Jones (2006)

In [ ]:
PJ41 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Dual/PJ2006_4_1.tsv"))
first(PJ41, 6)

In [ ]:
Dual = read_be(PJ41, :Cmax)

### 2S4P Designs

For 2S4P designs both RRTT|TTRR and RTRT|TRTR are supported

In [ ]:
PJ43 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/2S4P/PJ2006_4_3.tsv"))
first(PJ43, 6)

In [ ]:
Inner = read_be(PJ43)

In [ ]:
PJ44 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/2S4P/PJ2006_4_4.tsv"))
first(PJ44, 6)

In [ ]:
Outer = read_be(PJ44, :Cmax)

### Williams

Consider a 3 formulations Williams design

In [ ]:
PJ45 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Williams/PJ2006_4_5.tsv"))
first(PJ45, 6)

In [ ]:
W3F = read_be(PJ45)

Imagine for a moment that the reference formulation is actually S instead of R.
One can pass such a parameter as following.

In [ ]:
W3F = read_be(PJ45, reference = 'S')

Williams designs for four formulations are available as well

In [ ]:
PJ46 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Williams/PJ2006_4_6.tsv"))
first(PJ46, 6)

In [ ]:
W4F = read_be(PJ46)

### Additional documentation

#### Analyzing bioequivalence studies

In [ ]:
@doc BioequivalenceStudy

#### Generating study design data

In [ ]:
@doc generate_design

## References

Chow, Shein-Chung, and Jen-pei Liu. 2009. Design and Analysis of Bioavailability and Bioequivalence Studies. 3rd ed. Chapman & Hall/CRC Biostatistics Series 27. Boca Raton: CRC Press.

Clayton, D, and A Leslie. 1981. "The Bioavailability of Erythromycin Stearate versus Enteric-Coated Erythromycin Base When Taken Immediately before and after Food." Journal of International Medical Research 9 (6): 470-77. [DOI:10.1177/030006058100900608](https://doi.org/10.1177/030006058100900608).

Fuglsang, Anders, Helmut Schütz, and Detlew Labes. 2015. "Reference Datasets for Bioequivalence Trials in a Two-Group Parallel Design." The AAPS Journal 17 (2): 400-404. [DOI:10.1208/s12248-014-9704-6](https://doi.org/10.1208/s12248-014-9704-6).

Haidar, Sam H., Fairouz Makhlouf, Donald J. Schuirmann, Terry Hyslop, Barbara Davit, Dale Conner, and Lawrence X. Yu. 2008. "Evaluation of a Scaling Approach for the Bioequivalence of Highly Variable Drugs." The AAPS Journal 10 (3): 450-54. [DOI:10.1208/s12248-008-9053-4](https://doi.org/10.1208/s12248-008-9053-4).

Patterson, Scott D, and Byron Jones. 2006. Bioequivalence and Statistics in Clinical Pharmacology. Boca Raton: Chapman & Hall/CRC. ISBN: 9781420034936.

Schütz, Helmut, Detlew Labes, and Anders Fuglsang. 2014. "Reference Datasets for 2-Treatment, 2-Sequence, 2-Period Bioequivalence Studies." The AAPS Journal 16 (6): 1292-97. [DOI:10.1208/s12248-014-9661-0](https://doi.org/10.1208/s12248-014-9661-0).